In [ ]:
import pandas as pd

# Read the CSV file
weather_data = pd.read_csv("../data/open-meteo-63.43N10.40E12m.csv", skiprows=3)

# View the first 5 rows
weather_data.head()


In [ ]:
import requests
import pandas as pd
from pandasql import sqldf 
"""
Jeg måtte kjøre pip install pandasql i en egen notebook-celle
 for å få det til å funke, det gikk ikke å bare installere i terminal. 
 Jeg slettet den etterpå, men om det ikke funker for dere kan dere prøve det.
"""

In [3]:
"""Mitt forsøk på oppgave 2"""
"""
Hentet fra: https://api.met.no/ mer spesifikt hentet jeg API-en fra:
 https://api.met.no/weatherapi/locationforecast/2.0/documentation
"""


url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
# Denne API krever user agent, står på nettsiden
headers = {
    "User-Agent": "Nicole"
}

params = {
    "lat": 63.4305,
    "lon": 10.3951
}

res = requests.get(url, headers=headers, params=params)
data = res.json()

timeseries = data["properties"]["timeseries"] #Gir en liste av værdata per time 

weather_data = [
    {
        "time": entry["time"],
        "temperature": entry["data"]["instant"]["details"].get("air_temperature"),
        "humidity": entry["data"]["instant"]["details"].get("relative_humidity"),
        "wind_speed": entry["data"]["instant"]["details"].get("wind_speed"),
        "cloud_cover": entry["data"]["instant"]["details"].get("cloud_area_fraction"),
        "pressure": entry["data"]["instant"]["details"].get("air_pressure_at_sea_level"),
    }
    for entry in timeseries  # loop gjennom værmeldingen for hver time
]

df = pd.DataFrame(weather_data) # Gjør om til pandas dataframe (Gjør det til en tabell)

#Bruke pandasql for å finne 10 varmeste tidene (Den sto i oppgaven vi måtte bruke pandasql så jeg bare la noe til, vi kan endre på det senere)
query = "SELECT time, temperature FROM df WHERE temperature IS NOT NULL ORDER BY temperature DESC LIMIT 10"
result = sqldf(query)
print(result)

                   time  temperature
0  2025-04-16T13:00:00Z         19.0
1  2025-04-16T12:00:00Z         18.7
2  2025-04-16T11:00:00Z         18.6
3  2025-04-16T14:00:00Z         18.1
4  2025-04-16T10:00:00Z         17.0
5  2025-04-15T15:00:00Z         16.6
6  2025-04-16T15:00:00Z         16.5
7  2025-04-15T14:00:00Z         16.4
8  2025-04-15T16:00:00Z         16.2
9  2025-04-16T09:00:00Z         15.4


In [ ]:
"""Mitt forsøk på oppgave 3"""

def vis_manglende_verdier(df):
    """Finner og viser manglende verdier"""
    print("Manglende verdier per kolonne:")
    print(df.isnull().sum()) #Viser antall manglende verdier per kolonne (Det er ingen mangler :( vi må lage mangler, men tror det skal gå fint

def rens_data(df):
    """Renser manglende og 'rare' verdier"""
    #Lager en kopi så vi ikke endrer originalen
    df = df.copy()

    #Dette er eksempler på data rensing som jeg fant på nett vi kan finpusse sammen

    #Fjerner rader der temperaturen mangler
    df = df.dropna(subset=["temperature"]) 

    #Fyller NaN temperatur verdier med medianen
    df["temperature"] = df["temperature"].fillna(df["temperature"].mean())

    #Gjør det samme men med fuktighetsverdier
    df["humidity"] = df["humidity"].fillna(df["humidity"].mean())

    #Fjerner fuktighets verdier som ikke går an (over 100 eller under 0)
    df = df[(df["humidity"] >= 0) & (df["humidity"] <= 100)]

    #Erstatt ekstreme temperaturer som ikke er mulig/ gir mening med NaN
    df.loc[df["temperature"] < -60, "temperature"] = pd.NA
    df.loc[df["temperature"] > 60, "temperature"] = pd.NA

    return df

def formater_data(df):
    """Formaterer dataene for å gjøre dem mer strukturert og leselig"""
    df = df.copy()

    #Gjør tid kolonnen om til datetime
    df["time"] = pd.to_datetime(df["time"])

    #Runder verdier for lesbarhet
    df["temperature"] = df["temperature"].round(1)
    df["humidity"] = df["humidity"].round(0)
    df["wind_speed"] = df["wind_speed"].round(1)

    #Gir nye navn til kolonner om vi vil
    df.rename(columns={"wind_speed": "wind_speed_mps"}, inplace=True)

    #Endre rekkefølgen på kolonnene
    kolonne_rekkefølge = ["time", "temperature", "humidity", "wind_speed_mps"]
    df = df[[col for col in kolonne_rekkefølge if col in df.columns]]

    return df

def flagg_kuldegrader(df):
    """Legger til en ny kolonne for kuldegrader"""
    df = df.copy()
    df["Kuldegrader"] = [t < 0 if pd.notnull(t) else False for t in df["temperature"]]
    return df

def prepare_data(df):
    """Kombinerer alt"""
    vis_manglende_verdier(df)
    df = rens_data(df)
    df = formater_data(df)
    df = flagg_kuldegrader(df)
    return df

df_cleaned = prepare_data(df)

Manglende verdier per kolonne:
time           0
temperature    0
humidity       0
wind_speed     0
cloud_cover    0
pressure       0
dtype: int64
